<a href="https://colab.research.google.com/github/ohhhanna/twitter-sentiment-analysis/blob/main/sentiment_analysis_ff_GloVe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pickle
import re
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.cuda.empty_cache()

In [53]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


load the prepocessed data

In [54]:
x_train_file = open('/content/drive/My Drive/X_train.pkl', 'rb')
X_train = pickle.load(x_train_file)

x_test_file = open('/content/drive/My Drive/X_test.pkl', 'rb')
X_test = pickle.load(x_test_file)

y_train_file = open('/content/drive/My Drive/Y_train.pkl', 'rb')
Y_train = pickle.load(y_train_file)

y_test_file = open('/content/drive/My Drive/Y_test.pkl', 'rb')
Y_test = pickle.load(y_test_file)

x_val_file = open('/content/drive/My Drive/X_val.pkl', 'rb')
X_val = pickle.load(x_val_file)

y_val_file = open('/content/drive/My Drive/Y_val.pkl', 'rb')
Y_val = pickle.load(y_val_file)

#rb= read binary

VECTORIZATION WITH PRETRAINED EMBEDDING VECTORS USING GloVe

use of glove.6B.50d.text file,
has 400 words
represented by 50 dimension vectors


In [55]:
words = []
idx = 0
word2idx = {}
vectors = []

with open(f'/content/drive/MyDrive/Colab Notebooks/glove.6B.50d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        #we are just spliting the words and aooending them, giving them an index and embedding them with float64 and outting them in a vector

        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float64)

        vectors.append(vect)
#putting the vectors in glove
glove = {w: vectors[word2idx[w]] for w in words}

In [56]:
# Save GloVe dictionary.
glove_file = open("/content/drive/My Drive/glove_dict_50d.pkl", "wb")
pickle.dump(glove,glove_file)
glove_file.close()

# Load GloVe dictionary.
glove_file = open("/content/drive/My Drive/glove_dict_50d.pkl", "rb")
glove = pickle.load(glove_file)
# glove_file.close()

build a matrix of weights where shape will be equal to (dataset’s vocabulary length, word vectors dimension).

For each word in dataset’s vocabulary, check if it is on glove’s vocabulary. If there is, load its pre-trained word vector. Otherwise,initialize it with zeros. Finally,find the mean vector of each tweet

In [57]:
def Vectors_from_Glove(dataset):
  matrix_len = dataset['text'].str.split().str.len().sum()
  # print(matrix_len)
  words_found = 0
  words = 0
  mean_tweets = []

  for i,row in enumerate(dataset['text']):
    splited_words = str(row).split(' ')
    weight_matrix = np.zeros((len(splited_words),50))
    for j,word in enumerate(splited_words):
      words += 1
      if word in glove:
        weight_matrix[j] = glove[word]
        words_found += 1
    mean_tweets.append(np.mean(weight_matrix,axis=0))

  return np.array(mean_tweets), words_found, matrix_len

Print the ratio of words that founded in glove on train, validation and test set.

In [58]:
gloveX_train, words_found, matrix_len = Vectors_from_Glove(pd.DataFrame(X_train))
print(gloveX_train.shape)
# print(type(gloveX_train))
# print(len(gloveX_train[0]))
# print(gloveX_train[0])
print("Ratio of words from x_train that have founded in Glove: ", round(words_found/matrix_len,2))

gloveX_val, words_found, matrix_len = Vectors_from_Glove(pd.DataFrame(X_val))
print("Ratio of words from x_val that have founded in Glove: ", round(words_found/matrix_len,2))

gloveX_test, words_found, matrix_len = Vectors_from_Glove(pd.DataFrame(X_test))
print("Ratio of words from x_test that have founded in Glove: ", round(words_found/matrix_len,2))


(495012, 50)
Ratio of words from x_train that have founded in Glove:  0.93
Ratio of words from x_val that have founded in Glove:  0.92
Ratio of words from x_test that have founded in Glove:  0.93


**convert to tensors**

In [61]:
# Convert X datasets to tensors with function convert_to_tensor.
gloveX_train = torch.tensor(gloveX_train)
gloveX_val = torch.tensor(gloveX_val)
gloveX_test = torch.tensor(gloveX_test)

In [62]:
# Convert Y labels to tensors with torch.squeeze.
Y_train = torch.squeeze(torch.from_numpy(Y_train.to_numpy()).float())
Y_val = torch.squeeze(torch.from_numpy(Y_val.to_numpy()).float())
Y_test = torch.squeeze(torch.from_numpy(Y_test.to_numpy()).float())

In [63]:
#check cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [64]:

class Feed_forward_netModel(nn.Module):
    def __init__(self, input_dim, hidden_1, hidden_2, output_dim):
        super(Feed_forward_netModel, self).__init__()

        # First linear function
        self.layer_1 = nn.Linear(input_dim, hidden_1)
        # Non-linearity
        self.relu_1 = nn.ReLU()
        # Second linear function
        self.layer_2 = nn.Linear(hidden_1, hidden_2)
        # Non-linearity 2
        self.relu_2 = nn.ReLU()
        # Output linear layer
        self.layer_3 = nn.Linear(hidden_2, output_dim)

    def forward(self, x):
        out = self.layer_1(x)
        out = self.relu_1(out)
        out = self.layer_2(out)
        out = self.relu_2(out)
        out = self.layer_3(out)
        return torch.sigmoid(out)


In [65]:
# Dimensions of each layer and num of epochs.
input_dim = gloveX_train.shape[1]
hidden_dim_1 = 64
hidden_dim_2 = 16
output_dim = 1
num_epochs = 80

#define model
ff_model = Feed_forward_netModel(input_dim,hidden_dim_1,hidden_dim_2,output_dim)

# Define loss function.
criterion = nn.BCELoss()

# Define as optimizer Adam.
optimizer = optim.Adam(ff_model.parameters(),lr=1e-4,weight_decay=1e-4)

# Transfer all the computation to GPU (cuda device).
ff_model.to(device)
criterion = criterion.to(device)

In [66]:
#train model

def calculate_accuracy(y_true, y_pred):
  y_pred = torch.round(y_pred)
  correct = (y_true == y_pred).float()
  acc = correct.sum() / len(correct)
  return acc

batch_size = 10000

# Split train dataset to mini batches
X_train_mini_batches = torch.split(gloveX_train,batch_size)
Y_train_mini_batches = torch.split(Y_train,batch_size)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

# Start training
for epoch in range(num_epochs):
  epoch_loss = 0
  epoch_accuracy = 0
  validation_loss=0
  val_accuracy=0

  for X_train_mini_batch,Y_train_mini_batch in zip(X_train_mini_batches,Y_train_mini_batches):

    X_train_mini_batch = X_train_mini_batch.to(device)
    Y_train_mini_batch = Y_train_mini_batch.to(device)

    # Forward pass to get output
    train_prediction = ff_model.forward(X_train_mini_batch.float())
    train_prediction = torch.squeeze(train_prediction)

    # Calculate Loss
    train_loss = criterion(train_prediction,Y_train_mini_batch)

    # Clearing up accumulated gradients
    optimizer.zero_grad()

    # Getting gradients
    train_loss.backward()

    # Updating parameters
    optimizer.step()

    # Add each mini batch's loss
    epoch_loss += train_loss.item()

    # Add each mini batch's accuracy
    epoch_accuracy += calculate_accuracy(Y_train_mini_batch,train_prediction)

  # For some epochs print loss and accucary of train and validation set.
  if epoch % 1 == 0:

    gloveX_val = gloveX_val.to(device)
    Y_val = Y_val.to(device)

    # Forward pass to get output
    val_prediction = ff_model.forward(gloveX_val.float())
    val_prediction = torch.squeeze(val_prediction)

    # Calculate Loss
    val_loss = criterion(val_prediction,Y_val)

    # Add each mini batch's loss
    validation_loss += val_loss.item()

    # Add each mini batch's accuracy
    val_accuracy = calculate_accuracy(Y_val,val_prediction)

    epoch_loss /= len(X_train_mini_batches)
    epoch_accuracy /= len(X_train_mini_batches)
    val_losses.append(validation_loss)
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)
    val_accuracies.append(val_accuracy)

In [67]:
gloveX_test = gloveX_test.to(device)
Y_test = Y_test.to(device)

# Forward pass to get output
test_prediction = ff_model.forward(gloveX_test.float())
test_prediction = torch.squeeze(test_prediction)

# Add each mini batch's accuracy
test_accuracy = calculate_accuracy(Y_test,test_prediction)

print("Test Accuracy:",round(test_accuracy.item(),4)*100, "\n")

Test Accuracy: 68.64 



In [68]:
#Show the classification report
test_prediction = test_prediction.to(device)
test_prediction = test_prediction.ge(.5).view(-1).cpu()
Y_test = Y_test.cpu()

print(classification_report(Y_test,test_prediction))

              precision    recall  f1-score   support

         0.0       0.68      0.72      0.70      1281
         1.0       0.69      0.65      0.67      1219

    accuracy                           0.69      2500
   macro avg       0.69      0.69      0.69      2500
weighted avg       0.69      0.69      0.69      2500



making another pass with 150 epochs to see if accuracy changes

In [69]:
# Dimensions of each layer and num of epochs.
input_dim = gloveX_train.shape[1]
hidden_dim_1 = 64
hidden_dim_2 = 16
output_dim = 1
num_epochs = 150

#define model
ff2_model = Feed_forward_netModel(input_dim,hidden_dim_1,hidden_dim_2,output_dim)

# Define loss function.
criterion = nn.BCELoss()

# Define as optimizer Adam.
optimizer = optim.Adam(ff2_model.parameters(),lr=1e-4,weight_decay=1e-4)

# Transfer all the computation to GPU (cuda device).
ff2_model.to(device)
criterion = criterion.to(device)

In [70]:
def calculate_accuracy(y_true, y_pred):
    y_pred = torch.round(y_pred)
    correct = (y_true == y_pred).float()
    acc = correct.sum() / len(correct)
    return acc

batch_size = 10000

# Split train dataset to mini batches
X_train_mini_batches = torch.split(gloveX_train, batch_size)
Y_train_mini_batches = torch.split(Y_train, batch_size)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

# Start training
for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    validation_loss = 0
    val_accuracy = 0

    for X_train_mini_batch, Y_train_mini_batch in zip(X_train_mini_batches, Y_train_mini_batches):

        X_train_mini_batch = X_train_mini_batch.to(device)
        Y_train_mini_batch = Y_train_mini_batch.to(device)

        # Forward pass to get output
        train_prediction = ff2_model.forward(X_train_mini_batch.float())
        train_prediction = torch.squeeze(train_prediction)

        # Calculate Loss
        train_loss = criterion(train_prediction, Y_train_mini_batch)

        # Clearing up accumulated gradients
        optimizer.zero_grad()

        # Getting gradients
        train_loss.backward()

        # Updating parameters
        optimizer.step()

        # Add each mini batch's loss
        epoch_loss += train_loss.item()

        # Add each mini batch's accuracy
        epoch_accuracy += calculate_accuracy(Y_train_mini_batch, train_prediction)

    # For some epochs print loss and accuracy of train and validation set.
    if epoch % 1 == 0:

        gloveX_val = gloveX_val.to(device)
        Y_val = Y_val.to(device)

        # Forward pass to get output
        val_prediction = ff2_model.forward(gloveX_val.float())
        val_prediction = torch.squeeze(val_prediction)

        # Calculate Loss
        val_loss = criterion(val_prediction, Y_val)

        # Add each mini batch's loss
        validation_loss += val_loss.item()

        # Add each mini batch's accuracy
        val_accuracy = calculate_accuracy(Y_val, val_prediction)

        epoch_loss /= len(X_train_mini_batches)
        epoch_accuracy /= len(X_train_mini_batches)
        val_losses.append(validation_loss)
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)
        val_accuracies.append(val_accuracy)


In [71]:
gloveX_test = gloveX_test.to(device)
Y_test = Y_test.to(device)

# Forward pass to get output
test_prediction = ff2_model.forward(gloveX_test.float())
test_prediction = torch.squeeze(test_prediction)

# Add each mini batch's accuracy
test_accuracy = calculate_accuracy(Y_test,test_prediction)

print("Test Accuracy:",round(test_accuracy.item(),4)*100, "\n")

Test Accuracy: 70.32000000000001 



In [72]:
#Close the pickle files.
x_train_file.close()
y_train_file.close()
x_test_file.close()
y_test_file.close()
x_val_file.close()
y_val_file.close()